In [ ]:
! pip install kaggle

In [ ]:
# configuration the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# API to fetch yhe dataset from kaggle

!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#extracting a compressed file

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
# imporing the dependencies
import numpy as np
import pandas as pd

# re stand for regular expression
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# printng the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# DAta processing
# loading the data from csv file to pandas dataframe
twitter_data= pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [ ]:
# printing the first five rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns and reading the datset again
column_names= ['target','id','date','flag','user','tweet']
twitter_data= pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
# printing the first five rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# counting a number of missing value or text in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
tweet     0
dtype: int64

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
# convert the 4 to 1
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 means negative tweet and
1 means positive tweet

# stemming :- stemming is the proscces of reducing word to its keyword
example = actor, actress, acting
so here root word is act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content =re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['tweet'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,tweet,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# seprateding the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


# splitting the data to training data to test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape , X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['bad feel' 'love weekend' 'littlew bit far day trip fun get muddi today'
 ... 'twitter follow reform famou peopl'
 'aria cut hand open morn say like mommi anymor'
 'charlii sweet im excit aww miss u guy today']


In [ ]:
print(X_test)

['brodiejay oh im go wow mona vale real place afteral know suck mvill slow train pffft'
 'babi grow' 'paint black roll stone best' ...
 'belladonna miss good music' 'reec r know imma get'
 'ooooooohnoooooo forgot charg ipod work one sad littl bar left']


ML model dosent understand the textual data so we have to convert the string into numerical data


In [ ]:
# converting  textual data  into numerical data
vectorizer = TfidfVectorizer()
# TfidfVectorizer() convert the text data into numerical data
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)
# here 0 means words in the 1 tweet and 1 means words in the 2 tweet and so on

  (0, 129956)	0.6592576617698773
  (0, 31233)	0.7519171067329923
  (1, 437910)	0.8042773910733999
  (1, 241525)	0.5942540519072339
  (2, 411086)	0.18464287904169105
  (2, 278606)	0.4853139770765464
  (2, 145727)	0.16444046079173225
  (2, 140234)	0.22372378686138533
  (2, 416219)	0.2970321685021026
  (2, 93639)	0.16612263205577082
  (2, 128288)	0.28170394757881806
  (2, 42838)	0.22940430929074537
  (2, 237072)	0.6367700241935804
  (3, 286683)	0.3682809811482607
  (3, 8560)	0.37668420201866026
  (3, 445747)	0.2188627839232472
  (3, 179097)	0.4788299006944713
  (3, 308910)	0.41292329558269175
  (3, 201928)	0.524207488360723
  (4, 227660)	0.3291530007938817
  (4, 415818)	0.27525076285252903
  (4, 315347)	0.3291134109458034
  (4, 302738)	0.35966531233864635
  (4, 457632)	0.7604081439946883
  (5, 275098)	0.27168172484322706
  :	:
  (1279996, 150554)	0.1621318868138012
  (1279996, 286184)	0.19295841733643207
  (1279996, 445242)	0.1645728738109807
  (1279997, 336356)	0.6816932944012981
  (1279

In [ ]:
print(X_test)


  (0, 445807)	0.171511684458005
  (0, 428015)	0.34063344794032757
  (0, 414670)	0.1896315108355539
  (0, 387006)	0.16176032648819358
  (0, 371823)	0.20943732713604385
  (0, 334325)	0.18346549849682345
  (0, 317487)	0.1825714805413244
  (0, 313583)	0.3362151145199689
  (0, 298811)	0.13515991209574563
  (0, 280262)	0.4118569164941686
  (0, 272353)	0.3138965568980738
  (0, 220160)	0.12629294095566287
  (0, 177712)	0.12894291546244901
  (0, 149573)	0.1006209651184149
  (0, 53069)	0.38559379544960837
  (0, 5622)	0.30119901262778437
  (1, 154446)	0.7963195549499829
  (1, 30475)	0.6048761579069399
  (2, 384696)	0.5275602488060214
  (2, 344198)	0.45616935735592273
  (2, 306745)	0.4614836588658659
  (2, 43578)	0.434390421943326
  (2, 39327)	0.3345556683317226
  (3, 239066)	0.432753389073631
  (3, 218938)	0.5023157649783742
  :	:
  (319995, 49664)	0.7538991561657874
  (319996, 419143)	0.3618037526316817
  (319996, 408740)	0.4646424847685505
  (319996, 385819)	0.41302888896674317
  (319996, 35801

In [ ]:
# training the Machine Learning Model
# Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Model Evaluation
# accuracy_score on the training model
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy Score on the training data : ', training_data_accuracy)

Accuracy Score on the training data :  0.81003359375


In [ ]:
# accuracy_score on the test model
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy Score on the test data : ',test_data_accuracy)

Accuracy Score on the test data :  0.7783125


model Accuracy = 77.8 %

Savings the Trained Model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# using the saved model for future prediction

loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))


In [ ]:
X_new = X_test[300]
print(Y_test[300])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')

else:
  print('The given tweet is positive')

1
[1]
The given tweet is positive


In [ ]:
X_new = X_test[4]
print(Y_test[4])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')

else:
  print('The given tweet is positive')

0
[0]
Negative Tweet
